# The effect of homeless shelters on neighborhoods

In [329]:
import pandas as pd
import numpy as np 
import duckdb
import plotly.express 

## Data

[Police Precincts](https://www1.nyc.gov/site/nypd/bureaus/patrol/find-your-precinct.page)

[All Data](https://www1.nyc.gov/site/nypd/stats/crime-statistics/historical.page)

[DuckDB](https://duckdb.org/2021/05/14/sql-on-pandas.html)

In [330]:
shelters_data = pd.DataFrame({ # doesn't end up being used 
    "shelter_name" : ["Basic Housing", "Bay Family Center", "Win Shelter", "Park Savoy Hotel"],
    "date_opened" : ["08/21/2009", "09/30/2014", "09/30/2014", "6/30/2022"],
    "precinct" : [45, 61, 63, 18],
})
shelters_data["date_opened"] = pd.to_datetime(shelters_data["date_opened"])

In [331]:
seven_major_felonies = pd.read_excel("./data/seven-major-felony-offenses-by-precinct-2000-2021.xls", skiprows=[0, 1])
non_seven_major_felonies = pd.read_excel("./data/non-seven-major-felony-offenses-by-precinct-2000-2021.xls", skiprows=[0, 1])
misdemeanor_offenses = pd.read_excel("./data/misdemeanor-offenses-by-precinct-2000-2021.xls", skiprows=[0, 1])
violation_offenses = pd.read_excel("./data/violation-offenses-by-precinct-2000-2021.xls", skiprows=[0, 1])

In [332]:
seven_major_felonies_melted = seven_major_felonies.melt(
    id_vars=['CRIME', 'PCT'],
    var_name=['year'],
    value_name='count',
)
non_seven_major_felonies_melted = non_seven_major_felonies.melt(
    id_vars=['CRIME', 'PCT'],
    var_name=['year'],
    value_name='count',
)
misdemeanor_offenses_melted = misdemeanor_offenses.melt(
    id_vars=['CRIME', 'PCT'],
    var_name=['year'],
    value_name='count',
)
violation_offenses_melted = violation_offenses.melt(
    id_vars=['CRIME', 'PCT'],
    var_name=['year'],
    value_name='count',
)

In [333]:
# Just felonies
all_felonies = seven_major_felonies_melted.merge(non_seven_major_felonies_melted, how="outer")
total_felonies_per_year_melted = duckdb.query("""
    --begin-sql 
    select * from all_felonies where CRIME like 'TOTAL %'
    --end-sql
""").to_df().groupby(['PCT', 'year']).sum().reset_index()
total_felonies_per_year_melted["CRIME"] = 'TOTAL CRIMES'
all_felonies = all_felonies.merge(total_felonies_per_year_melted, how="outer")
all_felonies["year"] = pd.to_numeric(all_felonies["year"]) # Panderas test?

In [334]:
mean_felonies_per_year = all_felonies.groupby('year').mean().drop("PCT", axis=1).reset_index()

In [335]:
# Just offenses
all_offenses = misdemeanor_offenses_melted.merge(violation_offenses_melted, how="outer")
total_misdemeanors_per_year_melted = duckdb.query("""
    --begin-sql 
    select * from misdemeanor_offenses_melted where CRIME like 'TOTAL %'
    --end-sql
""").to_df().groupby(['PCT', 'year']).sum().reset_index()
total_misdemeanors_per_year_melted["CRIME"] = 'TOTAL CRIMES'
all_offenses = all_offenses.merge(total_misdemeanors_per_year_melted, how="outer")
all_offenses["year"] = pd.to_numeric(all_offenses["year"]) # Panderas test?

In [336]:
mean_offenses_per_year = all_offenses.groupby('year').mean().drop("PCT", axis=1).reset_index()

In [337]:
# sql alternative to the groupby above

agg_columns = ', '.join([f'''sum("{col}")''' for col in all_felonies.columns.to_list() if col not in ('PCT', 'CRIME')])
duckdb.query(f"""
    --begin-sql 
    select PCT, {agg_columns}
    from   all_felonies
    where  CRIME like 'TOTAL %'
    group by 1
    --end-sql
""").to_df()

,PCT,"sum(""year"")",sum(count)
0,1,132693.0,97184.0
1,5,132693.0,70794.0
2,6,132693.0,101374.0
3,7,132693.0,58066.0
4,9,132693.0,95858.0
...,...,...,...
72,115,132693.0,126996.0
73,120,132693.0,122232.0
74,121,132693.0,26686.0
75,122,132693.0,77440.0


## Plots

In [338]:
def graph_precinct_by_year(df: pd.DataFrame, PCT: int, year: int, crime_flavor: str):
    precinct_from_year = duckdb.query(f'''
        --begin-sql
        select *
        from   df
        where  CRIME = 'TOTAL CRIMES' and PCT = {PCT}
        --end-sql
    ''').to_df()
    all_time_avrg = df.groupby('year').mean().drop("PCT", axis=1).reset_index()
    all_time_avrg['PCT'] = 0
    df = precinct_from_year.merge(all_time_avrg, how="outer")

    fig = plotly.express.line(
        df,
        x="year",
        y="count",
        color='PCT',
        title=f'Precinct {PCT} {crime_flavor}'
    )
    fig.add_vline(year)
    fig.show()

In [339]:
fig = plotly.express.line(
    mean_felonies_per_year,
    x="year",
    y="count",
    title='Mean felonies'
)
fig.show()

In [340]:
fig = plotly.express.line(
    mean_offenses_per_year,
    x="year",
    y="count",
    title='Mean Offenses'
)
fig.show()

In [343]:
# Offense graphs

fig = plotly.express.line(
    duckdb.query(f'''
        --begin-sql
        select *
        from   all_offenses
        where  CRIME = 'TOTAL CRIMES'
        --end-sql
    ''').to_df(),
    x="year",
    y="count",
    color='PCT',
    title='Offenses By Precinct'
)
fig.show()

graph_precinct_by_year(all_offenses, 45, 2009, crime_flavor="Offenses")
graph_precinct_by_year(all_offenses, 61, 2014, crime_flavor="Offenses")
graph_precinct_by_year(all_offenses, 63, 2014, crime_flavor="Offenses")
graph_precinct_by_year(all_offenses, 18, 2022, crime_flavor="Offenses") 

In [344]:
# Felony graphs

fig = plotly.express.line(
    duckdb.query(f'''
        select *
        from   all_felonies
        where  CRIME = 'TOTAL CRIMES'
    ''').to_df(),
    x="year",
    y="count",
    color='PCT',
    title='Felonies By Precinct'
)
fig.update_layout(hovermode="x")
fig.show()

graph_precinct_by_year(all_felonies, 45, 2009, crime_flavor="Felonies")
graph_precinct_by_year(all_felonies, 61, 2014, crime_flavor="Felonies")
graph_precinct_by_year(all_felonies, 63, 2014, crime_flavor="Felonies")
graph_precinct_by_year(all_felonies, 18, 2022, crime_flavor="Felonies")